<a href="https://colab.research.google.com/github/JiyaVe/Movie_RecSys/blob/main/Movie_RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# numpy and scupy for maths
# lightfm for methods of recSys

import numpy as np
from lightfm.datasets import fetch_movielens
# movielens dataset contain movie ratings
from lightfm import LightFM

In [ ]:
#fetch data and format it

data = fetch_movielens(min_rating=4.0)#4 or higher rating
#it will create an interaction matrix from the csv file and
#store it in the data variable as a dictionary
#it also spilts the data into test and train

#print training and test data
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


In [ ]:
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 49906 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 5469 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1'),
 'item_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')}

In [ ]:
#create model
model = LightFM(loss = 'warp')
#loss function as "Weighted approximate-rank pairwise"


In [ ]:
#train model
model.fit(data['train'],epochs = 30,num_threads=2)
#data,no.of runs,no.of parallel runs

In [ ]:
#sample recsys
def sample_rec(model,data,user_ids):
  #no. of users and movies in train data
  n_users, n_items = data['train'].shape

  #generate rec for each user we input
  for user_id in user_ids:

    #movies they already like
    known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
    #lightfm makes problem binary 4 and less and 5 above

    #movies our model predicts they like
    scores = model.predict(user_id,np.arange(n_items))

    #rank them most to least liked
    top_items = data['item_labels'][np.argsort(-scores)]

    #print results
    print("User %s" % user_id)
    print("       Known Positives:")

    for x in known_positives[:3]:#top 3
      print("             %s"% x)

    print("       Recommended:")#top 3

    for x in top_items[:3]:
      print("               %s"% x)



In [ ]:
#call the function
sample_rec(model,data,[3,45,670])

User 3
       Known Positives:
             S
             C
             S
       Recommended:
               A
               T
               S
User 45
       Known Positives:
             T
             S
             W
       Recommended:
               C
               L
               E
User 670
       Known Positives:
             G
             G
             T
       Recommended:
               I
               S
               R


In [ ]:
from lightfm.evaluation import precision_at_k

- You can calculate the accuracy of the model using the precision_at_k function from lightfm.evaluation.
- Then, calculate the precision at k for the test set, setting k to any number you like (e.g., 10)
- This code will return the precision at k. It does this by looking at the top k predictions for each user and calculating what percentage of those were actually in the test set.


However, the low precision@k score suggests that while the overall ranking might be good, the model might not be great at identifying the very top recommendations within a short list (k=10).

In [ ]:
precision_at_k(model, data['test'], k=10).mean()

0.07815846

In [ ]:
from lightfm.evaluation import auc_score

In [ ]:
auc_score(model, data['test']).mean()

0.9174533